# Decoder evaluation

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import Pipeline, TextIteratorStreamer
from threading import Thread

class DecoderPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, text):
        # TODO: Prompt Template -> tokenizer.apply_chat_template
        if self.tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        return self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(self.device)

    def _forward(self, inputs):
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=200)
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()
        generated_text = ""
        for new_text in streamer:
            generated_text += new_text
        generated_text
        return generated_text

    def postprocess(self, outputs):
        outputs = outputs.replace(".\n", " ")
        outputs = outputs.replace("\n", "")
        return outputs

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [2]:
models = [
    "HuggingFaceH4/zephyr-7b-beta",
    "microsoft/Orca-2-13b",
    "mistralai/Mistral-7B-Instruct-v0.1",
    "Intel/neural-chat-7b-v3-1",
    "tiiuae/falcon-7b-instruct",
    #"meta-llama/Llama-2-7b-chat-hf", -> Config
    #"meta-llama/Llama-2-70b-chat-hf", -> Config
]

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

pbar1 = tqdm(total=len(models), position=0)
iterations = 500

for model_id in models:
    # Init model
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
    decoder = DecoderPipeline(model=model, tokenizer=tokenizer, device=device)

    # Test on SQuaDv2
    pbar2 = tqdm(total=500, position=1)
    for i in range(0, iterations):
        question = val[i]["question"]
        context = val[i]["context"]
        answer_list = val[i]["answers"].get("text")
        answer = "I don't know."
        if len(answer_list) > 0:
            answer = answer_list[0]
        
        pbar2.update(1)
    
    # clean up
    del model
    del tokenizer
    del decoder
    torch.cuda.empty_cache()
    pbar1.update(1)

100%|██████████| 3/3 [00:00<00:00,  5.47it/s]

100%|██████████| 500/500 [00:00<00:00, 922.27it/s]

100%|██████████| 500/500 [00:00<00:00, 117632.49it/s]

100%|██████████| 500/500 [00:00<00:00, 194324.68it/s]
